Code used to download daily stock price using python package yfinance. 
A file was created in google drive to store downloaded price on the initial run.
After initial run, only new data are downloaded, then append to the existing data. 

In [0]:
!pip install yfinance
import yfinance as yf
import pandas as pd
import numpy as np
import datetime as dt
import requests

  Created wheel for yfinance: filename=yfinance-0.1.54-py2.py3-none-any.whl size=22409 sha256=8b0e58548fcca87fac6367ee630540b5c6069460c8d8ac2c21241bfa276c7b07
  Stored in directory: /root/.cache/pip/wheels/f9/e3/5b/ec24dd2984b12d61e0abf26289746c2436a0e7844f26f2515c
Successfully built yfinance


In [0]:
### Uncomment this code when running the code first time to download the history data and create a dataset in google drive. Skip this step if history data is downloaded and file is created. 
# tickers = {'Chevron':'cvx', 'Google': 'goog', 'Amazon':'amzn', 'Facebook':'fb', 'Exxon':'xom', 'Tesla': 'tsla', 'Apple': 'aapl',
#            'Shell':'rds-a', 'BP':'bp', 'Microsoft':'msft','Honeywell':'hon','Gold':'gc=f','MYR':'myr=x', 
#            '10y Tresury':'^TNX', 'Brent':'bz=f', 'Copper':'hg=f', 'Lennar':'len', 'Phillips66':'psxp'}

# df_stocks=pd.DataFrame()
#   for key, value in tickers.items():
#     download=yf.download(value)
#     download = pd.DataFrame(download)
#     download['name'] = key
#     df_stocks=df_stocks.append(download)
# df_stocks.to_excel('google_drive_path/file_name.xlsx', sheet_name='Sheet1')

In [0]:
stocks=pd.read_excel('google_drive_path/file_name.xlsx', sheet_name='Sheet1') #Define start and end date for new download
start=stocks.groupby(['name'])['Date'].max()
start=pd.DataFrame(start)+ dt.timedelta(days=1)
end=dt.date.today()
print(start, end)

                  Date
name                  
10y Tresury 2020-03-24
Amazon      2020-03-24
Apple       2020-03-24
BP          2020-03-24
Brent       2020-03-24
Chevron     2020-03-24
Copper      2020-03-24
Exxon       2020-03-24
Facebook    2020-03-24
Gold        2020-03-24
Google      2020-03-24
Honeywell   2020-03-24
Lennar      2020-03-24
MYR         2020-03-24
Microsoft   2020-03-24
Phillips66  2020-03-24
Shell       2020-03-24
Tesla       2020-03-24 2020-03-25


In [0]:
#  Download new data from Yahoo Fiance
tickers = {'Chevron':'cvx', 'Google': 'goog', 'Amazon':'amzn', 'Facebook':'fb', 'Exxon':'xom', 'Tesla': 'tsla', 'Apple': 'aapl',
           'Shell':'rds-a', 'BP':'bp', 'Microsoft':'msft','Honeywell':'hon','Gold':'gc=f','MYR':'myr=x', 
           '10y Tresury':'^TNX', 'Brent':'bz=f', 'Copper':'hg=f', 'Lennar':'len', 'Phillips66':'psxp'}

if start['Date'].min() <= end: 
    df_stocks=pd.DataFrame()
    for key, value in tickers.items():            
        download=yf.download(value, start=start.loc[key][0].date(), end=end)
        download = pd.DataFrame(download)
        download['name'] = key
        df_stocks=df_stocks.append(download)
else:
    print('Already have latest data') 

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Clean up downloaded data. 
Today's data isn't always correct, so code below will remove today's data.
Sometimes, duplicate data are downloaded, code below will remove duplicates as well.

In [0]:
## Clean up downloaded data. 
if len(df_stocks)>0: 
  df_stocks=df_stocks.reset_index()
  df_stocks=df_stocks[df_stocks['Date']<pd.to_datetime('today').floor('d')]
if len(df_stocks)>0:
    df_stocks['Total']=df_stocks['High']+df_stocks['Volume']
    df_stocks=df_stocks[df_stocks['Total'] == df_stocks.groupby(['name', 'Date'])['Total'].transform('max')]
    df_stocks=df_stocks.drop('Total', axis=1)
    df_stocks=df_stocks.drop_duplicates()
    df_stocks=df_stocks.set_index('Date')
    stocks=stocks.set_index('Date')
    stocks=stocks.append(df_stocks, sort=False, ignore_index=False)      
    stocks.to_excel('google_drive_path/file_name.xlsx', sheet_name='Sheet1')

In [0]:
## Code below check if any ticker were missing from download data. 
if len(df_stocks)<len(tickers):
  missing_ticker=set(tickers.keys())-set(df_stocks['name'])
  print('Missing:', missing_ticker)
else:
  print('All Tickers download')

All Tickers download


In [0]:
# ## Code use for adding new ticker to the dataset
# ticker = 'psxp'
# name='Phillips66'  
# df_new=yf.download(ticker )
# df_new['name']=name
# df_new=df_new.reset_index()
# stocks=pd.read_excel('google_drive_path/file_name.xlsx', sheet_name='Sheet1')
# new_ticker=stocks.append(df_new)
# new_ticker=new_ticker.set_index('Date')
# with pd.ExcelWriter('google_drive_path/file_name.xlsx'',engine='openpyxl') as writer:
#     new_ticker.to_excel(writer)